In [ ]:
# ✅ Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Dataset path
DATASET_DIR = '/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X'

# ✅ Imports
import os
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, Add, GlobalAveragePooling2D, LayerNormalization, DepthwiseConv2D
from tensorflow.keras.optimizers import Adam
import random

# === CONFIG ===
IMG_SIZE = (224, 224)  # Standard for ConvNeXt
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

In [ ]:
# === ConvNeXt Building Blocks ===
def convnext_block(x, dim, drop_path=0.0, layer_scale_init_value=1e-6, name=None):
    """ConvNeXt block"""
    input_x = x
    
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=7, padding='same', name=f'{name}_dw_conv')(x)
    x = LayerNormalization(epsilon=1e-6, name=f'{name}_ln1')(x)
    
    # Pointwise convolution 1
    x = Conv2D(4 * dim, 1, name=f'{name}_pw_conv1')(x)
    x = Activation('gelu', name=f'{name}_gelu')(x)
    
    # Pointwise convolution 2
    x = Conv2D(dim, 1, name=f'{name}_pw_conv2')(x)
    
    # Layer scale
    if layer_scale_init_value > 0:
        x = x * layer_scale_init_value
    
    # Residual connection
    if drop_path > 0.0:
        # Simplified drop path (you can implement proper drop path if needed)
        pass
    
    x = Add(name=f'{name}_add')([input_x, x])
    return x

def downsample_layer(x, dim, name):
    """Downsampling layer"""
    x = LayerNormalization(epsilon=1e-6, name=f'{name}_ln')(x)
    x = Conv2D(dim, 2, strides=2, name=f'{name}_conv')(x)
    return x

In [ ]:
# === Visualize Sample Images ===
def plot_sample_images(dataset_path, num_images_per_class=5):
    classes = sorted(os.listdir(dataset_path))
    # plt.figure(figsize=(15, len(classes) * 2.5))
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        sample_images = random.sample(images, min(num_images_per_class, len(images)))
        for i, image_name in enumerate(sample_images):
            img_path = os.path.join(class_path, image_name)
            img = load_img(img_path, target_size=IMG_SIZE)
            # plt.subplot(len(classes), num_images_per_class, class_idx * num_images_per_class + i + 1)
            # plt.imshow(img)
            # plt.axis('off')
            # plt.title(class_name)
    # plt.tight_layout()
    # plt.show()

# === Load Dataset ===
def load_dataset(path):
    images = []
    labels = []
    for label_name in os.listdir(path):
        label_folder = os.path.join(path, label_name)
        if os.path.isdir(label_folder):
            for fname in os.listdir(label_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_folder, fname)
                    img = load_img(img_path, target_size=IMG_SIZE)
                    img = img_to_array(img) / 255.0
                    images.append(img)
                    labels.append(label_name)
    return np.array(images), np.array(labels)

In [ ]:
# === Load and Prepare Data ===
print("🔄 Loading dataset...")
X, y = load_dataset(DATASET_DIR)
print(f"✅ Loaded {len(X)} images.")

# === Visualize ===
print("🖼 Displaying sample images...")
plot_sample_images(DATASET_DIR)

In [ ]:
# === Encode Labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
loss_fn = 'binary_crossentropy'
activation_fn = 'sigmoid'
output_units = 1

# === Split Dataset ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [ ]:
# === Build ConvNeXt Model ===
def build_convnext_model(input_shape=(224, 224, 3), num_classes=1, depths=[3, 3, 9, 3], dims=[96, 192, 384, 768]):
    inputs = Input(shape=input_shape)
    
    # Stem
    x = Conv2D(dims[0], 4, strides=4, name='stem')(inputs)
    x = LayerNormalization(epsilon=1e-6, name='stem_ln')(x)
    
    # Stages
    for stage_idx, (depth, dim) in enumerate(zip(depths, dims)):
        # ConvNeXt blocks
        for block_idx in range(depth):
            x = convnext_block(x, dim, name=f'stage_{stage_idx}_block_{block_idx}')
        
        # Downsample (except for last stage)
        if stage_idx < len(depths) - 1:
            x = downsample_layer(x, dims[stage_idx + 1], name=f'downsample_{stage_idx}')
    
    # Global average pooling
    x = GlobalAveragePooling2D(name='gap')(x)
    x = LayerNormalization(epsilon=1e-6, name='head_ln')(x)
    
    # Fully connected layers
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    outputs = Dense(num_classes, activation=activation_fn, name='predictions')(x)
    
    model = Model(inputs, outputs, name='ConvNeXt')
    return model

# Create model (simplified version for faster training)
model = build_convnext_model(input_shape=(*IMG_SIZE, 3), num_classes=output_units, 
                           depths=[2, 2, 4, 2], dims=[64, 128, 256, 512])

# Compile
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=loss_fn, metrics=['accuracy'])
model.summary()

In [ ]:
# === Train Model ===
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

In [ ]:
# === Evaluate ===
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"📊 Test Accuracy: {test_acc:.2f}")

In [ ]:
# === Prediction Function ===
def predict_image(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    label = "Malignant" if prediction[0][0] > 0.5 else "Benign"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    print(f"Prediction: {label} ({confidence:.2f})")
    # plt.imshow(load_img(img_path))
    # plt.title(f"{label} ({confidence:.2f})")
    # plt.axis('off')
    # plt.show()

# Example:
# predict_image('/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X/benign/img001.png')